In [1]:
import numpy as np
import scanpy as sc
import cinemaot as co
import matplotlib.colors as colors
import matplotlib.pyplot as plt

import random
import torch
import sklearn
import os

from metrics import calculate_metrics

def set_seed(seed: int):
    # Set Python random seed
    random.seed(seed)

    # Set NumPy random seed
    np.random.seed(seed)

    # Set PyTorch random seed
    torch.manual_seed(seed)
    if torch.cuda.is_available():
        torch.cuda.manual_seed(seed)
        torch.cuda.manual_seed_all(seed)  # If using multi-GPU.

        # Ensure deterministic behavior in PyTorch (can slow down computations)
        torch.backends.cudnn.deterministic = True
        torch.backends.cudnn.benchmark = False

    # Set sklearn random seed
    sklearn.utils.check_random_state(seed)

    # Set environment variable for reproducibility
    os.environ['PYTHONHASHSEED'] = str(seed)

set_seed(123)

import warnings
warnings.filterwarnings("ignore")
warnings.filterwarnings("ignore", category=FutureWarning)

R_HOME is already set to: /vast/palmer/apps/avx2/software/R/4.3.2-foss-2022b-patched/lib64/R


Unable to determine R library path: Command '('/vast/palmer/apps/avx2/software/R/4.3.2-foss-2022b-patched/lib64/R/bin/Rscript', '-e', 'cat(Sys.getenv("LD_LIBRARY_PATH"))')' returned non-zero exit status 127.


/gpfs/gibbs/project/wang_zuoheng/xx244/Ndreamer/benchmark/metrics.py:29: DeprecationWarning: The global conversion available with activate() is deprecated and will be removed in the next major release. Use a local converter.
  anndata2ri.activate()
/home/xx244/.conda/envs/benchmark/lib/python3.11/site-packages/rpy2/robjects/pandas2ri.py:368: DeprecationWarning: The global conversion available with activate() is deprecated and will be removed in the next major release. Use a local converter.
  warnings.warn('The global conversion available with activate() '
/home/xx244/.conda/envs/benchmark/lib/python3.11/site-packages/rpy2/robjects/numpy2ri.py:241: DeprecationWarning: The global conversion available with activate() is deprecated and will be removed in the next major release. Use a local converter.
  warnings.warn('The global conversion available with activate() '


# ECCITE dataset

In [2]:
def evaluate_ECCITE(adata, embed):
    import warnings
    warnings.filterwarnings("ignore")
    warnings.filterwarnings("ignore", category=FutureWarning)

    print("Evaluating the performance of condition (whether or not perturbed) mixing, phase as cell type")
    # Prepare the environment
    import rpy2.robjects as robjects
    import anndata2ri
    anndata2ri.activate()
    library_path = "/gpfs/gibbs/project/wang_zuoheng/xx244/R/4.3/"  # Replace with the actual path
    robjects.r(f'.libPaths(c("{library_path}", .libPaths()))')
    calculate_metrics(adata, batch_key= 'perturbation', celltype_key="Phase", all=True, n_neighbors=15,
                      embed=embed)
    print("="*20)

    print("Evaluating the performance of condition (gene target of CRISPR) mixing, phase as cell type")
    # Prepare the environment
    import rpy2.robjects as robjects
    import anndata2ri
    anndata2ri.activate()
    library_path = "/gpfs/gibbs/project/wang_zuoheng/xx244/R/4.3/"  # Replace with the actual path
    robjects.r(f'.libPaths(c("{library_path}", .libPaths()))')
    calculate_metrics(adata, batch_key='gene_target', celltype_key="Phase", all=True, n_neighbors=15,
                      embed=embed)
    print("=" * 20)

    print("Evaluating the performance of batch mixing, phase as cell type")
    # Prepare the environment
    import rpy2.robjects as robjects
    import anndata2ri
    anndata2ri.activate()
    library_path = "/gpfs/gibbs/project/wang_zuoheng/xx244/R/4.3/"  # Replace with the actual path
    robjects.r(f'.libPaths(c("{library_path}", .libPaths()))')
    calculate_metrics(adata, batch_key='replicate', celltype_key="Phase", all=True, n_neighbors=15,
                      embed=embed)
    print("=" * 20)

In [3]:
model_names=["cinema_ot","mixscape","scCAPE","scgen"]
embed_names=['cf','ef','latent','latent']
for i in range(len(model_names)):
    model_name=model_names[i]
    print(model_name)
    adata=sc.read_h5ad("./"+model_name+"/ECCITE.h5ad")
    evaluate_ECCITE(adata, embed_names[i])

cinema_ot


Evaluating the performance of condition (whether or not perturbed) mixing, phase as cell type
LISI---


cLISI, bLISI, LISI_F1 0.9272810895608511 0.02052525161471544 0.04016153269706483


ASW---


asw_label 0.5390674099326134


...

Adding diffusion to step 7


Adding diffusion to step 8


kbet_score 0.14849862955136706
graph_conn 0.9386529420630577


NMI, ARI ---
NMI 0.3037855003992622
ARI 0.3258301475767778
positive and true positive rate---


        ASW_label      ARI       NMI   1-cLISI     bLISI  ASW_batch  \
latent    0.50607  0.32583  0.303786  0.915478  0.398578   0.962474   

        kBET Accept Rate  graph connectivity  PCR_batch  pos rate  \
latent          0.148499            0.938653        0.0  0.934681   

        true pos rate   F1 LISI  
latent       0.122099  0.555363  


# ASD dataset

In [4]:
def evaluate_ASD(adata, embed):
    import warnings
    warnings.filterwarnings("ignore")
    warnings.filterwarnings("ignore", category=FutureWarning)

    print("Evaluating the performance of condition (whether or not perturbed) mixing, phase as cell type")
    # Prepare the environment
    import rpy2.robjects as robjects
    import anndata2ri
    anndata2ri.activate()
    library_path = "/gpfs/gibbs/project/wang_zuoheng/xx244/R/4.3/"  # Replace with the actual path
    robjects.r(f'.libPaths(c("{library_path}", .libPaths()))')
    calculate_metrics(adata, batch_key='Perturbation', celltype_key="CellType", all=True, n_neighbors=15,
                      embed=embed)
    print("="*20)

    print("Evaluating the performance of condition (gene target of CRISPR) mixing, phase as cell type")
    # Prepare the environment
    import rpy2.robjects as robjects
    import anndata2ri
    anndata2ri.activate()
    library_path = "/gpfs/gibbs/project/wang_zuoheng/xx244/R/4.3/"  # Replace with the actual path
    robjects.r(f'.libPaths(c("{library_path}", .libPaths()))')
    calculate_metrics(adata, batch_key='perturb01', celltype_key="CellType", all=True, n_neighbors=15,
                      embed=embed)
    print("=" * 20)

    print("Evaluating the performance of batch mixing, phase as cell type")
    # Prepare the environment
    import rpy2.robjects as robjects
    import anndata2ri
    anndata2ri.activate()
    library_path = "/gpfs/gibbs/project/wang_zuoheng/xx244/R/4.3/"  # Replace with the actual path
    robjects.r(f'.libPaths(c("{library_path}", .libPaths()))')
    calculate_metrics(adata, batch_key="Batch", celltype_key="CellType", all=True, n_neighbors=15,
                      embed=embed)
    print("=" * 20)

In [5]:
model_names=["cinema_ot","mixscape","scCAPE","scgen"]
embed_names=['cf','ef','latent','latent']
for i in range(len(model_names)):
    model_name=model_names[i]
    print(model_name)
    adata=sc.read_h5ad("./"+model_name+"/ASD.h5ad")
    evaluate_ASD(adata, embed_names[i])

cinema_ot


Evaluating the performance of condition (whether or not perturbed) mixing, phase as cell type
Cell cluster Ependyma contains only one batch or has less than 10 cells. Skip.
Cell cluster Hippocampus contains only one batch or has less than 10 cells. Skip.
Cell cluster Fibroblast-like contains only one batch or has less than 10 cells. Skip.
Cell cluster Vascular contains only one batch or has less than 10 cells. Skip.
Cell cluster IPC contains only one batch or has less than 10 cells. Skip.
Cell cluster Doublet contains only one batch or has less than 10 cells. Skip.
Cell cluster Choroid Plexus contains only one batch or has less than 10 cells. Skip.
LISI---


cLISI, bLISI, LISI_F1 0.9972482982206406 0.06460152248153614 0.12134250456354069


ASW---


asw_label 0.5558530576527119


...

kBET---


0 labels consist of a single batch or is too small. Skip.


kbet_score 0.3484880657053233
graph_conn 0.9913409136260674


NMI, ARI ---
NMI 0.9465685765992654
ARI 0.9593014329988466
positive and true positive rate---


        ASW_label       ARI       NMI   1-cLISI     bLISI  ASW_batch  \
latent   0.532402  0.959301  0.946569  0.995901  0.179577   0.945594   

        kBET Accept Rate  graph connectivity  PCR_batch  pos rate  \
latent          0.348488            0.991341        0.0   0.98769   

        true pos rate   F1 LISI  
latent       0.071942  0.304287  
